# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt


# Import API key
from api_keys import geo_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bone,51.9669,12.1396,34.90,89,100,9.40,DE,1670171499
1,1,ushuaia,-54.8000,-68.3000,55.06,44,40,13.80,AR,1670171499
2,2,okha,53.5739,142.9478,-2.70,92,60,23.31,RU,1670171500
3,3,dikson,73.5069,80.5464,30.49,93,100,22.03,RU,1670171500
4,4,xambioa,-6.4111,-48.5364,80.51,83,96,4.70,BR,1670171501


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    s = city_data_df["Humidity"],
    color = "Country"
)
# Display the map plot
#plt.savefig("../output_data/map1.png")
hvplot.save(map_plot_1,'../output_data/map1.html')
map_plot_1




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] >= 75) & (city_data_df['Max Temp'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,port hedland,-20.3167,118.5667,80.65,57,1,9.22,AU,1670171527
86,86,abu dhabi,24.4667,54.3667,80.62,59,0,5.75,AE,1670171537
145,145,pochutla,15.7432,-96.4661,87.82,40,1,4.43,MX,1670171564
167,167,mar del plata,-38.0023,-57.5575,89.83,45,0,9.22,AR,1670171404
170,170,yulara,-25.2406,130.9889,79.27,17,0,7.14,AU,1670171574
177,177,lazaro cardenas,17.9583,-102.2000,85.12,53,0,2.30,MX,1670171577
182,182,acajutla,13.5928,-89.8275,86.45,51,0,5.77,SV,1670171579
203,203,emilio carranza,19.9711,-96.6114,77.95,70,0,6.49,MX,1670171590
341,341,chiredzi,-21.0500,31.6667,86.70,40,0,7.72,ZW,1670171651


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df['Hotel Name'] = ""
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,63,port hedland,-20.3167,118.5667,80.65,57,1,9.22,AU,1670171527,
86,86,abu dhabi,24.4667,54.3667,80.62,59,0,5.75,AE,1670171537,
145,145,pochutla,15.7432,-96.4661,87.82,40,1,4.43,MX,1670171564,
167,167,mar del plata,-38.0023,-57.5575,89.83,45,0,9.22,AR,1670171404,
170,170,yulara,-25.2406,130.9889,79.27,17,0,7.14,AU,1670171574,
177,177,lazaro cardenas,17.9583,-102.2000,85.12,53,0,2.30,MX,1670171577,
182,182,acajutla,13.5928,-89.8275,86.45,51,0,5.77,SV,1670171579,
203,203,emilio carranza,19.9711,-96.6114,77.95,70,0,6.49,MX,1670171590,
341,341,chiredzi,-21.0500,31.6667,86.70,40,0,7.72,ZW,1670171651,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius =10000
params = {
    "categories" :"accommodation.hotel",
    "apiKey" :geo_key,
    "limit" : 20,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
pochutla - nearest hotel: Hotel El Patio
mar del plata - nearest hotel: Nuevo Ostende
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
acajutla - nearest hotel: Hotel Lara
emilio carranza - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,63,port hedland,-20.3167,118.5667,80.65,57,1,9.22,AU,1670171527,The Esplanade Hotel
86,86,abu dhabi,24.4667,54.3667,80.62,59,0,5.75,AE,1670171537,Marriott Downtown Abu-Dhabi
145,145,pochutla,15.7432,-96.4661,87.82,40,1,4.43,MX,1670171564,Hotel El Patio
167,167,mar del plata,-38.0023,-57.5575,89.83,45,0,9.22,AR,1670171404,Nuevo Ostende
170,170,yulara,-25.2406,130.9889,79.27,17,0,7.14,AU,1670171574,Outback Pioneer Hotel & Lodge
177,177,lazaro cardenas,17.9583,-102.2000,85.12,53,0,2.30,MX,1670171577,Hotel Sol del Pacífico
182,182,acajutla,13.5928,-89.8275,86.45,51,0,5.77,SV,1670171579,Hotel Lara
203,203,emilio carranza,19.9711,-96.6114,77.95,70,0,6.49,MX,1670171590,No hotel found
341,341,chiredzi,-21.0500,31.6667,86.70,40,0,7.72,ZW,1670171651,Nesbitt Arms Hotel


In [10]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Nesbitt Arms Hotel',
    'street': 'Marula Drive',
    'city': 'Chiredzi',
    'county': 'Chiredzi',
    'state': 'Masvingo Province',
    'country': 'Zimbabwe',
    'country_code': 'zw',
    'lon': 31.666582,
    'lat': -21.0460724,
    'formatted': 'Nesbitt Arms Hotel, Marula Drive, Chiredzi, Zimbabwe',
    'address_line1': 'Nesbitt Arms Hotel',
    'address_line2': 'Marula Drive, Chiredzi, Zimbabwe',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Nesbitt Arms Hotel',
      'osm_id': 5347836376,
      'tourism': 'hotel',
      'osm_type': 'n'}},
    'distance': 435,
    'place_id': '512e1a321ea5aa3f40596d8b9c66cb0b35c0f00103f901d881c13e01000000920

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    s = hotel_df["Humidity"],
    hover_cols = ["Hotel Name","Country"],
    color = "City"
)
# Display the map plot
#plt.savefig("../output_data/map2.png")
hvplot.save(map_plot_2,'../output_data/map2.html')
map_plot_2




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Hotel Name,Country)